In [21]:
#!/usr/bin/env python3
import os
import zipfile
import openpyxl
import pyreqif.create
import pyreqif.reqif
import io
import openpyxl

file_names = ["BDY01.xlsx", "BDY02.xlsx", "BDY03.xlsx"]


for file_name in file_names:
    workbook = openpyxl.load_workbook(file_name)

    for sheet_name in workbook.sheetnames:
        worksheet = workbook[sheet_name]
        worksheet.row_dimensions.group(1, worksheet.max_row + 1, outline_level=0)
        empty_cols = [col_idx for col_idx, col in enumerate(worksheet.columns, 1) if all(cell.value is None for cell in col)]
        for col_idx in reversed(empty_cols):
            worksheet.delete_cols(col_idx, 1)
    # Save the workbook
    workbook.save(file_name)

    document_title, _ = os.path.splitext(os.path.basename(file_name))
    output_file = document_title + ".reqifz"

    #images = get_images_from_excel(file_name, output_file)
    wb = openpyxl.load_workbook(file_name)
    ws = wb.active

    columns = []
    for col_nr in range(1, ws.max_column + 1):
        columns.append(ws.cell(1, col_nr).value)

    document_reqif_id = "_{}ReqifId-Header".format(document_title)
    spec_reqif_id = "_{}ReqifId--spec".format(document_title)
    doc_type_ref = "_doc_type_ref"

    #
    # create doc:
    #
    mydoc = pyreqif.create.createDocument(document_reqif_id, title=document_title)
    pyreqif.create.addDocType(doc_type_ref, mydoc)
    # create primitive datatype
    #
    pyreqif.create.addDatatype("_datatype_ID", mydoc)
    #
    # create columns
    #
    for col in columns:
        pyreqif.create.addReqType("_some_requirement_type_id", "requirement Type", "_reqtype_for_" + col.replace(" ", "_"),col, "_datatype_ID", mydoc)
    #
    # create document hierarchy head
    #
    myHierarch = pyreqif.create.createHierarchHead(document_title, typeRef=doc_type_ref, id=spec_reqif_id)
    #
    # create child elements
    #
    #xls_req = {}
    hierarch_stack = []
    last_hierarch_element = myHierarch
    for row_nr in range(2, ws.max_row+1):
        xls_req = {}
        for x in range(1, ws.max_column + 1):
            column_name = columns[x - 1]
            cell_value = ws.cell(row_nr, x).value
            xls_req[column_name] = cell_value
        
        #GetId of test
        #if str(xls_req[col]) == "Object Identifier":
        
        #print(str(xls_req[columns[0]]))

        if "reqifId" not in xls_req:
            xls_req["reqifId"] = str(xls_req[columns[0]])+" - "+str(xls_req[columns[3]]) #+ pyreqif.create.creatUUID()
        
        for col in columns:
            #xls_req["reqifId"] = 'TEST'+ pyreqif.create.creatUUID()
            if type(xls_req[col]) == str:
                xls_req[col] = xls_req[col].replace(">", "&gt;")
                xls_req[col] = xls_req[col].replace("<", "&lt;")
            if xls_req[col] is not None:
                pyreqif.create.addReq(xls_req["reqifId"], "_some_requirement_type_id","<div>" + str(xls_req[col]) + "</div>", "_reqtype_for_" + col.replace(" ", "_"),mydoc)
        

        # do hierarchy
        hierarch_element = pyreqif.create.createHierarchElement(xls_req["reqifId"])
        level = ws.row_dimensions[row_nr].outline_level + 1
        if level > len(hierarch_stack):
            hierarch_stack.append(last_hierarch_element)
        elif level < len(hierarch_stack):
            hierarch_stack = hierarch_stack[0:level]

        if level <= len(hierarch_stack):
            current_head = hierarch_stack[level - 1]
            current_head.addChild(hierarch_element)
            last_hierarch_element = hierarch_element
        else:
            print(f"Warning: Skipping row {row_nr} due to invalid hierarchy level {level}")


    mydoc.hierarchy.append(myHierarch)

    #
    # safe reqif to string
    strIO = io.StringIO()
    pyreqif.reqif.dump(mydoc, strIO)

    # add reqifz as zip-file
    zip = zipfile.ZipFile(output_file, 'a', zipfile.ZIP_DEFLATED)
    zip.writestr(document_title + '.reqif', strIO.getvalue())

    zip.close()
    # write reqif string to file
    output_file_name = document_title + '.reqif'
    with open(output_file_name, 'w') as f:
        f.write(strIO.getvalue())

Sheet1
Sheet1
{'IDENTIFIER': '_doc_type_ref', 'LAST-CHANGE': '2023-03-14T09:11:51.991731', 'LONG-NAME': 'doc_type'}


c:\Users\gianpaolo.borrello\AppData\Local\Programs\Python\Python311\Lib\zipfile.py:1547: UserWarning: Duplicate name: 'BDY01.reqif'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


Sheet1
Sheet1
{'IDENTIFIER': '_doc_type_ref', 'LAST-CHANGE': '2023-03-14T09:11:51.991731', 'LONG-NAME': 'doc_type'}


c:\Users\gianpaolo.borrello\AppData\Local\Programs\Python\Python311\Lib\zipfile.py:1547: UserWarning: Duplicate name: 'BDY02.reqif'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


Sheet1
Sheet1
{'IDENTIFIER': '_doc_type_ref', 'LAST-CHANGE': '2023-03-14T09:11:51.991731', 'LONG-NAME': 'doc_type'}


c:\Users\gianpaolo.borrello\AppData\Local\Programs\Python\Python311\Lib\zipfile.py:1547: UserWarning: Duplicate name: 'BDY03.reqif'
  return self._open_to_write(zinfo, force_zip64=force_zip64)


In [ ]:
import openpyxl

file_name = r"C:\Automotive\STCReqManagement\pyreqif\BDY02__ - Copy.xlsx"
workbook = openpyxl.load_workbook(file_name)

for sheet_name in workbook.sheetnames:
    worksheet = workbook[sheet_name]
    worksheet.row_dimensions.group(1, worksheet.max_row + 1, outline_level=0)
    empty_cols = [col_idx for col_idx, col in enumerate(worksheet.columns, 1) if all(cell.value is None for cell in col)]
    for col_idx in reversed(empty_cols):
        worksheet.delete_cols(col_idx, 1)

# Save the workbook
workbook.save(file_name)


In [ ]:
pip install --upgrade openpyxl

In [3]:
#!/usr/bin/env python3
import pyreqif.rif
import pyreqif.html
import pyreqif.xlsx
import sys
import os

def process_file(filename):
    doc = pyreqif.rif.load(filename)
    basename = os.path.splitext(filename)[0]
    pyreqif.html.dump(doc, basename + ".html", os.path.dirname(filename))
    pyreqif.xlsx.dump(doc, basename + "_backgen.xlsx", os.path.dirname(filename))

def main():
    file_list = ["BDY01.reqif", "BDY02.reqif", "BDY03.reqif"]
    for filename in file_list:
        process_file(filename)

if __name__ == "__main__":
    main()